ОБЪЕДЕНЕНИЕ  ССР+ТДЦ+ГОД ВВОДА

In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import schedule
import time
import schedule
import time



# Загрузите файлы Excel
file1_path = r'T:\Лимиты\_Модель данных\01_Реестр ССР\01_Реестр ССР.xlsx'
file2_path = r'T:\Лимиты\_Модель данных\02_ТДЦ\02_ТДЦ.xlsx'
file3_path = r'T:\Лимиты\_Модель данных\05_Год ввода\05_Год ввода.xlsx'


# Чтение данных
sheet1_name = '01_Реестр'
df1 = pd.read_excel(file1_path, sheet_name=sheet1_name, usecols=['Тип', 'Подтип', 'Объект', '№','Идентификатор','Этап','Стадия','Состав','Наименование этапа','Статус',
                                                                 'Номер корректировки','Ключ корректировки','Ключ группы','Ключ этапа','UID','Заголовок заключения',
                                                                 'Срок экспертизы','Заключение','Дата составления сметы','Стоимость','Прогнозная стоимость',
                                                                 'Стоимость до корректировки','Утвержденная стоимость','Кроме того','Кроме того до корректировки','Утвержденное кроме того',
                                                                 'ПИР','ПИР до корректировки','Утвержденный ПИР','Смета','Смета до корректировки','Смета утвержденная'], header=1) #header=1 заголовок со второй строки
# Чтение второго файла
df2 = pd.read_excel(file2_path, usecols=['Ключ этапа', 'ТДЦ'], header=1) #header=1 заголовок со второй строки
df3 = pd.read_excel(file3_path, usecols=['Ключ этапа', 'Год ввода','Примечание'], header=1) #header=1 заголовок со второй строки

# Удаление дублей в df3,df2 по столбцу 'Ключ этапа'
df3_unique = df3.drop_duplicates(subset='Ключ этапа')                   #Удаление дубликатов из 5_Год ввода
df2_unique = df2.drop_duplicates(subset='Ключ этапа')                   #Удаление дубликатов из 02_ТДЦ

# Сброс индекса
df1.reset_index(drop=True, inplace=True)                                 #Сброс индекса удаляет порядковые номера созданные автоматически


# Объединение данных
SSR = df1.merge(df2_unique, how='left', on='Ключ этапа')                 #Объединение ССР и ТДЦ
SSR = SSR.merge(df3_unique, how='left', on='Ключ этапа')                 #Объединение ССР и 05_Год ввода

# SSR['Год ввода'] = pd.to_datetime(SSR['Год ввода'], errors='coerce')
# SSR['Год ввода'] = SSR['Год ввода'].dt.strftime("%Y")




# Если в ТДЦ ноль или пусто то сумма подтягиваетсяиз столбца утверженная стоимость
# Заменяем пустые строки на NaN
SSR['ТДЦ'].replace('', np.nan, inplace=True)                             #Замена пустых ячеек на Nan для обработки
# Теперь заменяем NaN значения
SSR.loc[pd.isna(SSR['ТДЦ']), 'ТДЦ'] = SSR.loc[SSR['Утвержденная стоимость'] > 0, 'Утвержденная стоимость']
SSR.loc[SSR['ТДЦ'] == 0, 'ТДЦ'] = SSR.loc[SSR['Утвержденная стоимость'] > 0, 'Утвержденная стоимость']


#Фильтрация данных
SSR = SSR[SSR['Статус'].str.contains('Положительное', na=False)]         #Выборка только в статусе положительное
SSR = SSR[SSR['Состав'].str.contains('ПС' and 'С', na=False)]            #Выборка только по составу ПС и С

# Сохранение данных в новый файл
with pd.ExcelWriter(r'T:\BIM\99. Прочее\Хохулин\SSR+TDC+GOD VVODA.xlsx') as writer:
    SSR.to_excel(writer, sheet_name="Aggregation", index=False)          #Сохранение файла




ОСНОВНОЙ 09_ГРД

In [ ]:
import pandas as pd
import os
from datetime import date
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm
from tqdm import tqdm
import re


# Путь к папке с файлами Excel
# folder_path = r'T:\Лимиты\_Модель данных\09_ГРД\новая форма\Временый'
folder_path = r'T:\Лимиты\_Модель данных\09_ГРД'


# Исключаемые файлы и папки
excluded_files = ['.XLSX.XLSX', 'ТЕСТ.XLSX']
excluded_folders = ['_Управление', 'Архив', 'новая форма']

# Список для хранения DataFrame
combined_dfs = []

# Укажите столбцы, которые хотите сохранить как строки (dtype_map и str "str тип данных" соблюдение формата данных как в исходниках)
dtype_map = {

    'Номер документа': str,
    'Стадия': str,
    'Ревизия': str,
    'Номер изменения': str,
    'Тип документа': str,
    'Проект': str,
    'Этап': str,
    'Объект': str,
    'Подобъект': str,
    'Получение документа от Разработчика - Дата (План)': str,
    'Получение документа от Разработчика - Дата (Прогноз)': str,
    'Получение документа от Разработчика - Дата (Факт)': str,
    'Статус': str,
    'Статус Заказчика': str,
    'Статус МИП': str,
    'Документ передан в МИП в окончательной редакции в бумажном виде – Дата (План)': str,
    'Документ передан в МИП в окончательной редакции в бумажном виде – Дата (Прогноз)': str,
    'Выдача документа в производство работ - Дата (Факт) (временный)': str,
    'Марка': str,
    'Период': str,
    'Веб ссылка': str

}

def make_links_non_clickable(text):
    """Преобразует ссылки в обычный текст"""
    if pd.isna(text):
        return text

    text = str(text)
    # # Заменяем http:// и https:// на пустую строку
    text = text.replace('http://', '').replace('https://', '')
    # Удаляем специальные символы, которые могут сделать текст кликабельным
    text = text.replace('[', '').replace(']', '').replace('<', '').replace('>', '')
    return text

# Обработка файлов
for filename in os.listdir(folder_path):
    if filename.endswith('.XLSX'):
        file_path = os.path.join(folder_path, filename)
        try:
            # Чтение данных
            combined_df = pd.read_excel(file_path, usecols=list(dtype_map.keys()), dtype=dtype_map)

            # Обработка столбца с ссылками
            if 'Веб ссылка' in combined_df.columns:
                combined_df['Веб ссылка'] = combined_df['Веб ссылка'].apply(make_links_non_clickable)

            combined_dfs.append(combined_df)

        except Exception as e:
            print(f"Ошибка при обработке файла {filename}: {str(e)}")


# # Перебор всех файлов в папке
# for filename in os.listdir(folder_path):
#     if filename.endswith('.XLSX'):
#         file_path = os.path.join(folder_path, filename)
#         try:
#             # Чтение данных из файла Excel
#             combined_df = pd.read_excel(file_path, usecols=list(dtype_map.keys()), dtype=dtype_map)#,skiprows=0
#             # Убираем специальные символы из столбца 'Веб ссылка'
#             # combined_df['Веб ссылка'] = combined_df['Веб ссылка'].replace(r'[^\w\s]', '', regex=True)

#             # Добавляем DataFrame в список
#             combined_dfs.append(combined_df)
#         except Exception as e:
#             print(f"Ошибка при чтении {filename}: {e}")

# # Объединяем все DataFrame в один, если это нужно
# final_df = pd.concat(combined_dfs, ignore_index=True)




# Объединение всех DataFrame в один
combined_df = pd.concat(combined_dfs, ignore_index=True) #ПЛАН/ФАКТ ОБЩИЙ

# 2. Создаем копию столбца 'Ревизия_1' перед преобразованием
combined_df['Ревизия_дубль'] = combined_df['Ревизия'].copy(deep=True)

#Переименовываем столбцы
combined_df = combined_df.rename(columns={'Ревизия': 'Ревизия_1'})
combined_df = combined_df.rename(columns={'Ревизия_дубль': 'Ревизия'})

#Чтение данных из другого файла со второй строки с листа '00.02_Объекты'
additional_file_path = r'T:\Лимиты\_Модель данных\00_Объекты\00_Объекты.xlsx'
additional_combined_df = pd.read_excel(additional_file_path, sheet_name='00.02_Объекты', skiprows=1, header=0)

#Чтение данных из другого файла со второй строки с листа '02_ТДЦ' из файла '05_Год ввода'
additional_file_path_1 = r'T:\Лимиты\_Модель данных\05_Год ввода\05_Год ввода.xlsx'
additional_combined_df_1 = pd.read_excel(additional_file_path_1, sheet_name='02_ТДЦ', skiprows=1, header=0)


# Фильтрация по столбцу "Стадия"
combined_df = combined_df[combined_df['Стадия'] == 'Рабочая документация']
# # Удаление пустых столбцов
# combined_df = combined_df.dropna(axis=1, how='all')
# Удаление строк, где "Номер документа" пустой
combined_df = combined_df[combined_df['Номер документа'].notna()]


#Факт даты по столбцам
# Предполагаем, что даты разделены запятыми или другим разделителем, это может быть изменено по необходимости
dates_split = combined_df['Выдача документа в производство работ - Дата (Факт) (временный)'].str.split(';', expand=True)
# Переименуем столбцы
dates_split.columns = [f'Производство_факт.{i+1}' for i in range(dates_split.shape[1])]
# Объединяем новый DataFrame с исходным
combined_df = pd.concat([combined_df, dates_split], axis=1)


#Преобразование столбца к типу datetime так-как dtype_map и str тип данных
combined_df['Получение документа от Разработчика - Дата (План)'] = pd.to_datetime( combined_df['Получение документа от Разработчика - Дата (План)'], format='%d.%m.%Y', errors='coerce' )
combined_df['Документ передан в МИП в окончательной редакции в бумажном виде – Дата (План)'] = pd.to_datetime(combined_df['Документ передан в МИП в окончательной редакции в бумажном виде – Дата (План)'], format='%d.%m.%Y', errors='coerce')
combined_df['Документ передан в МИП в окончательной редакции в бумажном виде – Дата (Прогноз)'] = pd.to_datetime(combined_df['Документ передан в МИП в окончательной редакции в бумажном виде – Дата (Прогноз)'],format='%d.%m.%Y', errors='coerce')
combined_df['Выдача документа в производство работ - Дата (Факт) (временный)'] = pd.to_datetime(combined_df['Выдача документа в производство работ - Дата (Факт) (временный)'], format='%d.%m.%Y', errors='coerce')
combined_df['Получение документа от Разработчика - Дата (План)'] = pd.to_datetime(combined_df['Получение документа от Разработчика - Дата (План)'], format='%d.%m.%Y', errors='coerce')
combined_df['Получение документа от Разработчика - Дата (Прогноз)'] = pd.to_datetime(combined_df['Получение документа от Разработчика - Дата (Прогноз)'], format='%d.%m.%Y', errors='coerce')
combined_df['Производство_факт.1'] = pd.to_datetime(combined_df['Производство_факт.1'], format='%d.%m.%Y', errors='coerce')




# Создание нового столбца "В производство_План_общий" на основе "Документ передан в МИП в окончательной редакции в бумажном виде – Дата (План)" # Если значение пустое, заполняем его значением "Получение документа от Разработчика - Дата (План)" + 14 дней
combined_df['В производство_План_общий'] = combined_df['Документ передан в МИП в окончательной редакции в бумажном виде – Дата (План)'].fillna( combined_df['Получение документа от Разработчика - Дата (План)'] + pd.DateOffset(days=14) )
# Если после этого есть пропуски, заполняем их датой 01.01.1970
combined_df['В производство_План_общий'] = combined_df['В производство_План_общий'].fillna(pd.Timestamp('1970-01-01'))

# Создание нового столбца "Прогноз" на основе "Документ передан в МИП в окончательной редакции в бумажном виде – Дата (Прогноз)" # Если значение пустое, заполняем его значением "В производство_План_общий"
combined_df['Прогноз+план'] = combined_df['Документ передан в МИП в окончательной редакции в бумажном виде – Дата (Прогноз)'].fillna( combined_df['В производство_План_общий'])
combined_df['Прогноз+план'] = combined_df['Прогноз+план'].fillna(pd.Timestamp('1970-01-01'))

# # Инициализируем столбец 'Факт' значениями NaN
# combined_df['Факт нет даты'] = np.nan
# combined_df.loc[combined_df['Статус'] == 'Передан «В производство работ»', 'Факт нет даты'] = combined_df['Выдача документа в производство работ - Дата (Факт) (временный)']

#'Сравнение даты Факт/План' Факт < тек даты а План > тек даты
# Создаем текущую дату
current_date = datetime.now()
# Условие для создания нового столбца
combined_df['Сравнение даты Факт/План'] = 0  # Изначально присваиваем 0
# Применяем условия
combined_df.loc[
    (combined_df['Производство_факт.1'] < current_date) &
    (combined_df['В производство_План_общий'] > current_date) &
    (combined_df['Статус'] == 'Передан «В производство работ»'),
    'Сравнение даты Факт/План'] = 1

#Если хотябы один раз в столбце 'Сравнение даты Факт/План' по шифру втретилось 1 то по остальным проставляется тоже в столбце 'Маркер факт<план'
# Создаем новый столбец и заполняем его значениями из 'Сравнение даты Факт/План'
combined_df['Маркер факт<план'] = combined_df['Сравнение даты Факт/План']
# Создаем маску для проверки, есть ли хотя бы один 1 в 'Сравнение даты Факт/План' по 'Номеру документа'
mask = combined_df.groupby('Номер документа')['Сравнение даты Факт/План'].transform(lambda x: 1 if 1 in x.values else 0)
# Обновляем новый столбец, проставляя 1 для записей со значением 0, если есть хотя бы один 1
combined_df.loc[(combined_df['Маркер факт<план'] == 0) & (mask == 1), 'Маркер факт<план'] = 1

# Создание нового столбца Факт и по статусу документа (Передан «В производство работ») проставляем 1 если соответствует статусу и 0 если нет
combined_df['Факт счет'] = np.where(combined_df['Статус'] == 'Передан «В производство работ»', 1, 0)

# # Сначала создайте текущую дату
# current_date = pd.to_datetime(datetime.now().date())
# # Создайте новый столбец и присвойте 1, если значение в 'В производство_План_общий' меньше или равно текущей дате, иначе 0
# combined_df['Меньше или равно текущая дата'] = (combined_df['В производство_План_общий'] <= current_date).astype(int)


# Извлекает текст слева в столбце Проект до символа - и убирает пробелы у извлеченного текста
combined_df['СТДО'] = combined_df['Проект'].str.split('-').str[0].str.strip()

# Извлекает текст слева в столбце Марка до символа - и убирает пробелы у извлеченного текста
combined_df['Марка_Описание'] = combined_df['Марка'].str.split('-').str[0].str.strip()



# Фильтрация по столбцу статус исключает позиции
combined_df = combined_df[combined_df['Статус'] != 'Аннулирован']
combined_df = combined_df[combined_df['Статус'] != 'Заменен']
combined_df = combined_df[combined_df['Статус'] != 'Отозван']

# Фильтрация по столбцу статус исключает позиции
combined_df = combined_df[combined_df['Ревизия_1'] != 'З']
combined_df = combined_df[combined_df['Ревизия_1'] != 'П']
combined_df = combined_df[combined_df['Ревизия_1'] != 'П.01']
combined_df = combined_df[combined_df['Ревизия_1'] != 'П.01.1']
combined_df = combined_df[combined_df['Ревизия_1'] != 'П.01.4']
combined_df = combined_df[combined_df['Ревизия_1'] != 'П.02']
combined_df = combined_df[combined_df['Ревизия_1'] != 'П.03']
combined_df = combined_df[combined_df['Ревизия_1'] != 'П.04']
combined_df = combined_df[combined_df['Ревизия_1'] != 'П.05']
combined_df = combined_df[combined_df['Ревизия_1'] != 'П.06']
combined_df = combined_df[combined_df['Ревизия_1'] != 'П.07']
combined_df = combined_df[combined_df['Ревизия_1'] != 'П01']
combined_df = combined_df[combined_df['Ревизия_1'] != 'А1']
combined_df = combined_df[combined_df['Ревизия_1'] != 'А.1']
combined_df = combined_df[combined_df['Ревизия_1'] != 'А']

#Фильтрация по столбцу Тип документа исключает позиции
combined_df = combined_df[combined_df['Тип документа'] != 'ГРФ - График/ План мероприятий']
combined_df = combined_df[combined_df['Тип документа'] != 'ИДП - Исходные данные для проектирования']
combined_df = combined_df[combined_df['Тип документа'] != 'ЛСР - Локальный сметный расчет']
combined_df = combined_df[combined_df['Тип документа'] != 'ЛРС - Локальный ресурсный сметный расчет']
combined_df = combined_df[combined_df['Тип документа'] != 'ОЛ - Опросный лист']
combined_df = combined_df[combined_df['Тип документа'] != 'РВИ - Разрешение на внесение изменений']
combined_df = combined_df[combined_df['Тип документа'] != 'СО - Спецификация оборудования, изделий, материалов']
combined_df = combined_df[combined_df['Тип документа'] != 'СОГЛ - Согласование']
combined_df = combined_df[combined_df['Тип документа'] != 'ТУ - Технические условия']
combined_df = combined_df[combined_df['Тип документа'] != 'ГРД - Главный реестр документации']


#Исключение из списка шифров с имеющим данный текст.
exclude_terms = ['Тест','Архив'] #.ОЛ','-СО-', '.ССО','МИП.И',
# Фильтрация DataFrame
combined_df = combined_df[~combined_df['Номер документа'].str.contains('|'.join(exclude_terms), na=False)]

#Исключение из списка шифров с имеющим на конце данный текст.
exclude_terms  = ['\-СО$','\.СО$', '\.ССО$', '-СО-$', '\-СО-$', '-СО $', '\-СО $', '\.ОЛ$', '^МИП\.И$', '^МИП.И', ' ОП$', '\-ОГЗ$'] #'^ССО$',
# Фильтрация DataFrame
combined_df = combined_df[~combined_df['Номер документа'].str.contains('|'.join(exclude_terms), regex=True, na=False)]

#ПОСЛЕДНЯЯ Ревизия
# Преобразуем 'Ревизия_1' в формат, который позволяет корректно сравнивать
combined_df['Ревизия_1'] = pd.to_numeric(combined_df['Ревизия_1'].str.replace('.', ''), errors='coerce')
# Группируем по 'Номер документа' и находим последние значения в 'Ревизия_1'
combined_df['Максимальная Ревизия по изм'] = combined_df.groupby('Номер документа')['Ревизия_1'].transform('max')
# Условия для нового столбца
def status_condition(row):
    if row['Статус'] == 'Передан «В производство работ»':
        return 0
    return 1
# Применяем условие к каждой строке
combined_df['Последняя Ревизия по изм'] = combined_df.apply(status_condition, axis=1)

#Максимальное значение Ревизия по Номер документа и проставляет 1 если максимальное и 0 если нет
combined_df['Последняя Ревизия маркер'] = combined_df.groupby('Номер документа')['Ревизия_1'].transform(lambda x: (x == x.max()).astype(int))

#Если хотябы один раз в столбце 'Последняя Ревизия маркер' по шифру втретилось 1 то по остальным проставляется тоже в столбце 'Последняя Ревизия маркер'
# Создаем новый столбец и заполняем его значениями из 'Последняя Ревизия маркер'
combined_df['Маркер последней ревизии по изм'] = combined_df['Последняя Ревизия маркер']
# Создаем маску для проверки, есть ли хотя бы один 1 в 'Последняя Ревизия маркер' по 'Номеру документа'
mask = combined_df.groupby('Номер документа')['Последняя Ревизия маркер'].transform(lambda x: 1 if 1 in x.values else 0)
# Обновляем новый столбец, проставляя 1 для записей со значением 0, если есть хотя бы один 1
combined_df.loc[(combined_df['Маркер последней ревизии по изм'] == 0) & (mask == 1), 'Маркер последней ревизии по изм'] = 1


# Ревизия
# Если хотябы один раз в столбце 'Детализация ревизия' по шифру втретилось 1 то по остальным проставляется тоже в столбце 'Детализация ревизия' это если в столбце не 1 или 0 а цифры от 1-го и выше берет максимальное значение
# 1. Создаем столбец 'Детализация ревизия' на основе маркера
combined_df['Детализация ревизия'] = np.where(
    combined_df['Маркер последней ревизии по изм'] == 1,
    combined_df['Ревизия'],
    '0'  # Используем строковый ноль
)

# 2. Для каждого номера документа находим ревизии, где маркер = 1
def get_marked_revisions(group):
    marked = group[group['Маркер последней ревизии по изм'] == 1]['Ревизия']
    return marked.iloc[0] if not marked.empty else '0'

marked_revisions = combined_df.groupby('Номер документа').apply(get_marked_revisions)

# 3. Проставляем найденные значения всем строкам с тем же номером документа
combined_df['Детализация ревизия текст'] = combined_df['Номер документа'].map(marked_revisions)

# 4. Заменяем '0' на пустую строку в текстовом представлении
combined_df['Детализация ревизия текст'] = combined_df['Детализация ревизия текст'].replace('0', '')

# 5. Для столбца 'Детализация ревизия' оставляем только маркированные ревизии
combined_df['Детализация ревизия'] = np.where(
    combined_df['Маркер последней ревизии по изм'] == 1,
    combined_df['Ревизия'],
    ''
)



#НЕ РАВЗРАБОТАНО ИМИП
#Создание нового столбца Не разработано ИМИП счет и по статусу документа (Передан «В производство работ») проставляем 1 если соответствует статусу и 0 если нет
combined_df['Не разработано ИМИП счет'] = np.where(combined_df['Статус'] == 'Не выпущен', 1, 0)

#РАССМОТРЕНИИ (в т.ч. ДСМ)
#Создание нового столбца и по статусу документа и статусу заказчика (Передан «В производство работ» и т.д.) проставляем 1 если соответствует статусу и 0 если нет
valid_statuses = ['На рассмотрении', 'Согласован', 'Не рассматривался', 'Согласован с замечаниями',
                  'Получен', 'Утвержден «В производство работ»']
# Условие, при котором значение будет равно 1
condition = (combined_df['Статус'].isin(valid_statuses)) & \
            (combined_df['Статус Заказчика'] == 'На рассмотрении у Заказчика')
# Присваиваем 1, если условие выполнено, иначе 0
combined_df['На рассмотрении (в т.ч. ДСМ)'] = np.where(condition, 1, 0)

#Возврат ИМИП
#Создание нового столбца возврат ИМИП и по статусу документа (Не согласован») проставляем 1 если соответствует статусу и 0 если нет
combined_df['Возврат ИМИП счет'] = np.where(combined_df['Статус'] == 'Не согласован', 1, 0)


#ИЗМЫ
#Функция для извлечения текста справа от последнего разделителя (',', '.', '-')
def extract_number_right(change_number):
    if pd.isna(change_number):  # Проверка на пустое значение
        return None  # Или пустую строку ''
#Ищем справа от последнего разделителя
    match = re.search(r'([^,.-]*)$', change_number)
    if match:
# Удаляем пробелы с левой и правой сторон
        return match.group(1).strip()
# Если разделителей нет, возвращаем исходную строку
    return change_number.strip()
# Применяем функцию к столбцу 'Номер изменения'
combined_df['Номер изма'] = combined_df['Номер изменения'].apply(extract_number_right)

#СЧЕТ ИЗМОВ
combined_df['Номер изма'] = pd.to_numeric(combined_df['Номер изма'].str.replace('.', ''), errors='coerce')
def пометить_max_и_оставить_ноль(combined_df):
    # Заменяем пустые значения в 'Номер изма' на -1, чтобы они не влияли на поиск максимума
    combined_df['Номер изма'].fillna(-1, inplace=True)
    # Находим максимальное значение 'Номер изма' для каждой группы 'Номер документа'
    max_izm_per_document = combined_df.groupby('Номер документа')['Номер изма'].transform('max')
    # Создаем столбец 'Маркер последней ревизии по изм', который равен 1, если 'Номер изма' равно максимальному значению
    combined_df['Маркер последней ревизии по изм'] = np.where(combined_df['Номер изма'] == max_izm_per_document, 1, 0)
    # Восстанавливаем пустые значения в 'Номер изма' обратно
    combined_df.loc[combined_df['Номер изма'] == -1, 'Маркер последней ревизии по изм'] = 0
combined_df = pd.DataFrame(combined_df)
пометить_max_и_оставить_ноль(combined_df)

# Меняем -1 на 0
combined_df['Номер изма'] = combined_df['Номер изма'].replace(-1, 0)

#Если 'Маркер последней ревизии по изм' = 1 то берутся данные из 'Номер изменения' иначе 0
# Функция для определения значения на основе условия
def marker_revision(row):
    if row['Маркер последней ревизии по изм'] == 1:
        return row['Номер изменения']
    else:
        return 0
# Применение функции и создание нового столбца
combined_df['Изм детализация'] = combined_df.apply(marker_revision, axis=1)


#Если хотябы один раз в столбце 'Изм детализация' по шифру втретилось 1 то по остальным проставляется тоже в столбце 'Изм детализация'
# Создаем новый столбец и заполняем его значениями из 'Изм детализация'
combined_df['Изм детализация текст'] = combined_df['Изм детализация']
# Создаем маску для проверки, есть ли хотя бы один 1 в 'Изм детализация' по 'Номеру документа'
mask = combined_df.groupby('Номер документа')['Изм детализация'].transform(lambda x: 'Изм.1' if 'Изм.1' in x.values else 0)
# Обновляем новый столбец, проставляя 1 для записей со значением 0, если есть хотя бы один 1
combined_df.loc[(combined_df['Изм детализация текст'] == 0) & (mask == 'Изм.1'), 'Изм детализация текст'] = 'Изм.1'


#Создание столбца и по статусу передано в ПР и Последняя Ревизия маркер = 0 то 1 иначе 0
valid_statuses = ['Передан «В производство работ»']
# Условие, при котором значение будет равно 1
condition = (combined_df['Статус'].isin(valid_statuses)) & \
            (combined_df['Последняя Ревизия маркер'] == 0)
# Присваиваем 1, если условие выполнено, иначе 0
combined_df['Маркер_ПР_не последняя ревизия'] = np.where(condition, 1, 0)

#Если хотябы один раз в столбце ''Маркер_ПР_не последняя ревизия' по шифру втретилось 1 то по остальным проставляется тоже в столбце 'Маркер факт<план'
# Создаем новый столбец и заполняем его значениями из 'Маркер_ПР_не последняя ревизия'
combined_df['Макс ПР не последняя ревизия'] = combined_df['Маркер_ПР_не последняя ревизия']
# Создаем маску для проверки, есть ли хотя бы один 1 в 'Маркер_ПР_не последняя ревизия' по 'Номеру документа'
mask = combined_df.groupby('Номер документа')['Маркер_ПР_не последняя ревизия'].transform(lambda x: 1 if 1 in x.values else 0)
# Обновляем новый столбец, проставляя 1 для записей со значением 0, если есть хотя бы один 1
combined_df.loc[(combined_df['Макс ПР не последняя ревизия'] == 0) & (mask == 1), 'Макс ПР не последняя ревизия'] = 1

#Столбец фильтр для детализаии по условиям
def filter_detail(row):
    if row['Последняя Ревизия маркер'] == 1 and row['Маркер_ПР_не последняя ревизия'] == 0:
        return 1
    elif (row['Последняя Ревизия маркер'] == 1 and
          row['Маркер_ПР_не последняя ревизия'] == 1 and
          row['Статус'] == "Передан «В производство работ»"):
        return 2
    elif (row['Последняя Ревизия маркер'] == 0 and
          row['Маркер_ПР_не последняя ревизия'] == 1 and
          row['Статус'] == "Передан «В производство работ»"):
        return 3
    else:
        return 0
# Применение функции к каждому ряду DataFrame
combined_df['Фильтр для детализации'] = combined_df.apply(filter_detail, axis=1)



# СБОРКА И ОБРАБОТКА ОБЪЕКТОВ
# Объединение по ключевому столбцу из файла 00_Объекты
combined_df = combined_df.merge(additional_combined_df[['СТДО', 'Объект']], on='СТДО', how='left')
# # Создает столбец Объекты1 Определяет в столбце значения только ГЛВК и Ленинградский вокзал и объединяет их
# combined_df['Объекты1'] = ''
# # Создаем маску для условий
# mask = (combined_df['СТДО'] == 'ГЛВК') & (combined_df['Объект_y'] == 'Ленинградский вокзал')
# # Объединяем значения только для выбранных строк
# combined_df.loc[mask, 'Объекты1'] = combined_df.loc[mask, 'СТДО'].astype(str) + " " + combined_df.loc[mask, 'Объект_y'].astype(str)
# # Меняем пустое значение на nan
# combined_df['Объекты1'] = combined_df['Объекты1'].replace('', np.nan)
# # Заполняем пустые значения (замененные на nan) в Объекты1 из Объект_y
# combined_df['Объекты'] = combined_df['Объекты1'].fillna( combined_df['Объект_y'])
#Выполняем слияние (left join), используя столбцы с разными именами
#merge с объктами 00.02_Объекты (лист) Полный ключ
combined_df = combined_df.merge(additional_combined_df[['Объект', 'Полный ключ']],left_on='Объект_y',right_on='Объект',how='left')
combined_df['Полный ключ']
#Удаляем столбцы 'СТДО', 'Объект_y', 'Объекты1'
# combined_df.drop(columns=['СТДО', 'Объект_y','Объекты1','Объект'], inplace=True)

#Переименовываем столбец
combined_df = combined_df.rename(columns={'Объект': 'Объекты'})

# Объединение по ключевому столбцу из файла 00_Объекты
combined_df = combined_df.merge(additional_combined_df[['Полный ключ', 'Тип']], on='Полный ключ', how='left')


#ИЗВЛЕЧЕНИЕ НОМЕРА ЭТАПА ИЗ СТОЛБЦА ЭТАП СТРОИТЕЛЬСТВА
def extract_stage_number(stage_string):
    # (Код функции, как описано выше)
    if not isinstance(stage_string, str):
        return None  # Возвращаем None, если входное значение не строка
    match = re.match(r'Этап ([\d.]+)', stage_string)  # Ищем "Этап " за которым следует одна или более цифр и точек
    if match:
         return match.group(1).rstrip('.')  # Возвращаем найденные цифры (номер этапа) без последней точки
    else:
        return stage_string   # Возвращаем None, если соответствие не найдено или stage_string чтобы вернуть исходную строку
#Применяем функцию к столбцу 'Номер этапа'
combined_df['Номер этапа'] = combined_df['Этап'].apply(extract_stage_number)
# Объединяем значения только для выбранных строк
combined_df['Ключ этапа'] = combined_df['Полный ключ'].astype(str) + '_ОП_' + combined_df['Номер этапа'].astype(str)
# Замена ключа
combined_df['Ключ этапа'] = combined_df['Ключ этапа'].replace('ПГС_НКЦ_ОП_1', 'ПГС_НКЦ_ПП_1')
#Замена в номере документа
combined_df['Ключ этапа'] = combined_df['Ключ этапа'].replace('13-4002-СЛ-Р-03-ПPC', '13-4002-СЛ-Р-03-ПРС')

#Фильтрация по столбцу Номер документа исключает позиции
combined_df = combined_df[combined_df['Номер документа'] != '21-4001-Р-16Э-ТКР4']
combined_df = combined_df[combined_df['Номер документа'] != 'Печать из архива (МТЛН)']
combined_df = combined_df[combined_df['Номер документа'] != 'тест иерархии']
combined_df = combined_df[combined_df['Номер документа'] != 'Примыкание к ст."Сокольники" Сокольнической линии метрополитена. Архитектурные работы']
combined_df = combined_df[combined_df['Номер документа'] != 'Документация']
combined_df = combined_df[combined_df['Номер документа'] != 'Материалы']
combined_df = combined_df[combined_df['Номер документа'] != 'Досмотровая']
combined_df = combined_df[combined_df['Номер документа'] != 'Конструкция']
combined_df = combined_df[combined_df['Номер документа'] != 'Лист']
combined_df = combined_df[combined_df['Номер документа'] != 'Научно-проектная документация']
combined_df = combined_df[combined_df['Номер документа'] != 'План']
combined_df = combined_df[combined_df['Номер документа'] != 'Планы']
combined_df = combined_df[combined_df['Номер документа'] != 'ПОЯСНИТЕЛЬНАЯ']
combined_df = combined_df[combined_df['Номер документа'] != 'проект изменения']
combined_df = combined_df[combined_df['Номер документа'] != 'Расположение оборудования']
combined_df = combined_df[combined_df['Номер документа'] != 'Спецификация']
combined_df = combined_df[combined_df['Номер документа'] != 'Станция "Сокольники"']
combined_df = combined_df[combined_df['Номер документа'] != 'Технологический регламент']
combined_df = combined_df[combined_df['Ключ этапа'] != 'ПГС_НКЦ_ПП_1']


# Удаление строк, где "Объект_x" пустой
combined_df = combined_df[combined_df['Объект_x'].notna()]


#Из год ввода подтягиваем дату (год) и пустые ячейки заполняем н/д
combined_df = combined_df.merge(additional_combined_df_1[['Ключ этапа', 'Год ввода']],left_on='Ключ этапа',right_on='Ключ этапа',how='left')
combined_df['Год ввода'] = combined_df['Год ввода'].fillna('н/д')

#Год ввода факт <Текущего года введено остальное подтягивает из столбца Год ввода
# Текущий год
current_year = datetime.now().year
# Создание нового столбца "Факт год ввода"
def determine_fact_year(row):
    try:
        year = int(row['Год ввода'])
        if year < current_year:
            return "Введено"
        else:
            return str(year) # Или просто year, если тип должен быть int
    except (ValueError, TypeError):
        return row['Год ввода']  # Возвращаем исходное значение, если не число или NaN
combined_df['Факт год ввода'] = combined_df.apply(determine_fact_year, axis=1)

#Удаление столбцов
combined_df.drop(columns=['Объект_y'], inplace=True) #'Объекты1','Объект'
# combined_df.drop(columns=['Полный ключ'], inplace=True)
combined_df.drop(columns=['Производство_факт.2', 'Производство_факт.3','Производство_факт.4','Производство_факт.5','Производство_факт.6','Производство_факт.7'], inplace=True)

#Убираем 'nan_ОП_' из столбца Ключ Этапа
# Удаляем строки, где "ключ этапа" содержит "nan_ОП_"
combined_df = combined_df[~combined_df['Ключ этапа'].str.contains('nan_ОП_', na=False)]


# combined_df['полная ссылка'] = 'https://' + combined_df['Веб ссылка']


#ВНЕ ГРД собирает данные из файлов добавляет столбец ВНЕ ГРД в ПЛАН_ФАКТ_ОБЩИЙ и делает Сравнение двух столбцов если в ГРД есть значения из листа Институт то 0 в противном случае 1
# Путь к первой папке с файлами Excel
folder_path_1 = r'T:\BIM\3. Систематизация\01_Проекты\01_ИБ Реестр ССР\05_Тест\ГРД\Пример ГРД'

# Список для хранения DataFrame из первой папки
combined_df_1 = []

# Перебор всех файлов в первой папке
for file_name in os.listdir(folder_path_1):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path_1, file_name)
        # Чтение данных из файла Excel (начиная с 5-й строки и листа ГРД_РД)
        df = pd.read_excel(file_path, sheet_name='ГРД_РД', header=4,
                           usecols=['Обозначение комплекта', 'Наименование комплекта', 'Проект'])
        combined_df_1.append(df)

# Объединение всех DataFrame в один для первой папки
final_df_1 = pd.concat(combined_df_1, ignore_index=True)

# Удаление строк, где "Обозначение комплекта" пустой и "Проект" равен 21
final_df_1 = final_df_1[final_df_1['Обозначение комплекта'].notna()]
final_df_1 = final_df_1[final_df_1['Проект'] != 21]

#Сравнение двух столбцов если в ГРД есть значения из листа Институт то 0 в противном случае 1
# Создаем множество эталонных значений для точного сравнения
grd_values = set(final_df_1['Обозначение комплекта'].dropna().astype(str))
    # Добавляем столбец с правильной логикой:
    # 0 - если номер документа есть в ГРД (точное совпадение)
    # 1 - если номера нет в ГРД
combined_df['Вне ГРД'] = combined_df['Номер документа'].apply(
        lambda x: '0' if str(x) in grd_values else '1'
    )

# #Подтягивает столбец Проект из листа Институт (final_df_2) в ПЛАН_ФАКТ_ОБЩИЙ
# # Создаем словарь для соответствия значений
# project_mapping = final_df_1.set_index('Обозначение комплекта')['Проект'].to_dict()
# # Подтягиваем значения проекта в combined_df
# combined_df['Проект'] = combined_df['Номер документа'].map(project_mapping)
# # Для несовпадающих значений можно указать заполнитель (по желанию)
# combined_df['Проект'] = combined_df['Проект'].fillna('Не указан')  # или другой заполнитель


# Запись объединенного DataFrame в новый файл Excel
with pd.ExcelWriter(r'T:\Лимиты\_Модель данных\09_ГРД\_Управление\09_ГРД.xlsx') as writer:
    combined_df.to_excel(writer, sheet_name="ПЛАН_ФАКТ_ОБЩИЙ", index=False)
    final_df_1.to_excel(writer, sheet_name="Институт", index=False)






#Удаляем столбцы 'Полный ключ', 'Номер этапа'
# combined_df.drop(columns=['Полный ключ', 'Номер этапа'], inplace=True)

# #ВСЕГО ИЗМОВ
# #Создание нового столбца и по статусу документа 'На рассмотрении', 'Передан «В производство работ»', 'Возврат ИМИП' проставляем 1 если соответствует статусу и 0 если нет
# valid_statuses = ['На рассмотрении', 'Передан «В производство работ»', 'Не согласован']
# # Условие, при котором значение будет равно 1
# condition = (combined_df['Статус'].isin(valid_statuses))
# # Присваиваем 1, если условие выполнено, иначе 0
# combined_df['Всего ИЗМ'] = np.where(condition, 1, 0)

# #ВЫДАНО В ПР ИЗМОВ
# #Создание нового столбца и по статусу документа 'На рассмотрении', 'Передан «В производство работ»', 'Возврат ИМИП' проставляем 1 если соответствует статусу и 0 если нет
# valid_statuses = ['Передан «В производство работ»']
# # Условие, при котором значение будет равно 1
# condition = (combined_df['Статус'].isin(valid_statuses))
# # Присваиваем 1, если условие выполнено, иначе 0
# combined_df['Выдано в ПР ИЗМ'] = np.where(condition, 1, 0)


# # Допустим, у вас уже есть ваш DataFrame combined_df
# valid_statuses = ['На рассмотрении', 'Согласован', 'Не рассматривался',
#                   'Согласован с замечаниями', 'Получен', 'Утвержден «В производство работ»']
# condition = (combined_df['Статус'].isin(valid_statuses))
# # Присваивание нового столбца
# combined_df['На рассмотрении (в т.ч. ДСМ) ИЗМ'] = np.where(condition, 1, 0)


# #РАССМОТРЕНИИ (в т.ч. ДСМ_1)
# #Создание нового столбца и по статусу документа и статусу заказчика (Передан «В производство работ» и т.д.) проставляем 1 если соответствует статусу и 0 если нет
# valid_statuses = ['На рассмотрении', 'Согласован', 'Не рассматривался', 'Согласован с замечаниями',
#                   'Получен','Утвержден «В производство работ»']
# # Условие, при котором значение будет равно 1
# condition = (combined_df['Статус'].isin(valid_statuses)) & \
#             (combined_df['Статус Заказчика'] == 'На рассмотрении у Заказчика')
# # Присваиваем 1, если условие выполнено, иначе 0
# combined_df['На рассмотрении (в т.ч. ДСМ) ИЗМ_1'] = np.where(condition, 1, 0)


# #ВОЗВРАТ ИМИП ИЗМОВ
# #Создание нового столбца и по статусу документа 'На рассмотрении', 'Передан «В производство работ»', 'Возврат ИМИП' проставляем 1 если соответствует статусу и 0 если нет
# valid_statuses = ['Не согласован']
# # Условие, при котором значение будет равно 1
# condition = (combined_df['Статус'].isin(valid_statuses))
# # Присваиваем 1, если условие выполнено, иначе 0
# combined_df['Возврат ИМИП ИЗМ'] = np.where(condition, 1, 0)


# # Функция для извлечения текста слева до первого любого знака
# def extract_number(change_number):
#     if pd.isna(change_number):  # Проверка на пустое значение
#         return None  # Можно вернуть `None`, или пустую строку `''`
#     match = re.match(r'([^,.-]*)', change_number)
#     if match:
#         return match.group(1).strip()  # Удаляем пробелы с левой и правой сторон
#     return change_number.strip()

# # Применяем функцию к столбцу 'Номер изменения'
# combined_df['Извлеченный номер'] = combined_df['Номер изменения'].apply(extract_number)


# # Объединяем столбцы 'СТДО' и 'Объект_y' в новый столбец 'Объекты'
# combined_df['Объекты'] = combined_df['СТДО'] + " " + combined_df['Объект_y']



c:\Users\Khokhulin.AS\PROJECT_4\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\Khokhulin.AS\PROJECT_4\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\Khokhulin.AS\AppData\Local\Temp\ipykernel_15128\2059587633.py:285: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  marked_revisions = combined_df.groupby('Номер документа').apply(get_marked_revisions)
C:\Users\Khokhulin.AS\AppData\Local\Temp\ipykernel_15128\2059587633.py:340: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series t

ПЕРЕНОС ФАЙЛОВ ГРД ПУТЕМ КОПИРОВАНИЯ ИЗ ПАПКИ В ПАПКУ ДЛЯ ГРД

In [ ]:
import os
import shutil
from datetime import datetime

# Определение пути к исходной и целевой папкам
source_folder = r'T:\Лимиты\_Модель данных\09_ГРД'  # Замените на свой путь
destination_folder_base = r'T:\Лимиты\_Модель данных\09_ГРД\Архив'  # Замените на свой путь

# Исключаемые файлы и папки
excluded_files = ['.XLSX.XLSX', 'ТЕСТ.XLSX']
excluded_folders = ['_Управление', 'Архив', 'новая форма']

# Получение текущей даты
current_date = datetime.now().strftime('%Y-%m-%d')
destination_folder = os.path.join(destination_folder_base, current_date)

# Создание целевой папки, если она не существует
os.makedirs(destination_folder, exist_ok=True)

# Копирование файлов Excel
for filename in os.listdir(source_folder):
    if filename.endswith('.XLSX') or filename.endswith('.XLSX'):
        source_file = os.path.join(source_folder, filename)
        destination_file = os.path.join(destination_folder, filename)
        shutil.copy2(source_file, destination_file)  # Копируем файл
        # os.remove(source_file)  # Удаляем оригинал
        print(f'Скопирован файл: {filename}')

print('Все файлы успешно скопированы и перенесены.')


Копирует целые папки в архивную папку с датой, сохраняя структуру

In [1]:
import os
import shutil
from datetime import datetime
import logging

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='folder_copy.log'
)

def copy_folders_with_backup(source_folders, destination_base):
    """
    Копирует целые папки в архивную папку с датой, сохраняя структуру
    :param source_folders: список исходных папок для копирования
    :param destination_base: базовая папка для архива
    """
    try:
        # Создаем папку архива с текущей датой
        current_date = datetime.now().strftime('%Y-%m-%d')
        destination_folder = os.path.join(destination_base, current_date)

        os.makedirs(destination_folder, exist_ok=True)
        logging.info(f"Создана папка архива: {destination_folder}")

        total_copied = 0

        for source_folder in source_folders:
            if not os.path.exists(source_folder):
                logging.warning(f"Исходная папка не существует: {source_folder}")
                continue

            # Создаем подпапку в архиве с именем исходной папки
            folder_name = os.path.basename(os.path.normpath(source_folder))
            dest_subfolder = os.path.join(destination_folder, folder_name)

            try:
                # Копируем всю папку рекурсивно
                shutil.copytree(
                    src=source_folder,
                    dst=dest_subfolder,
                    dirs_exist_ok=True  # Для Python 3.8+
                )
                total_copied += 1
                logging.info(f"Скопирована папка: {source_folder} -> {dest_subfolder}")

            except shutil.Error as copy_error:
                logging.error(f"Ошибка при копировании {source_folder}: {str(copy_error)}")
            except Exception as e:
                logging.error(f"Неожиданная ошибка: {str(e)}")

        logging.info(f"Готово! Скопировано папок: {total_copied}")
        return True

    except Exception as e:
        logging.critical(f"Критическая ошибка: {str(e)}")
        return False

# Пути для обработки (можно добавить сколько угодно папок)
source_folders = [
    r'T:\Лимиты\_Модель данных\00_Объекты',
    r'T:\Лимиты\_Модель данных\01_Реестр ССР',
    r'T:\Лимиты\_Модель данных\02_ТДЦ',
    r'T:\Лимиты\_Модель данных\Прогноз'
    r'T:\Лимиты\_Модель данных\04_График МГЭ',
    r'T:\Лимиты\_Модель данных\05_Год ввода',
    r'T:\Лимиты\_Модель данных\06_Лимиты',
    r'T:\Лимиты\_Модель данных\07_Приемка актов',
    r'T:\Лимиты\_Модель данных\08_Статус прохождения МГЭ',
    r'T:\Лимиты\_Модель данных\09_ГРД',
]

destination_base = r'T:\BIM\99. Прочее\Хохулин\Лимиты Архив'

# Запуск копирования
if copy_folders_with_backup(source_folders, destination_base):
    print("Операция завершена успешно. Подробности в folder_copy.log")
else:
    print("Возникли ошибки при копировании. Проверьте folder_copy.log")

Операция завершена успешно. Подробности в folder_copy.log


ОБНОВЛЕНИЕ ФАЙЛА ССР

In [2]:
import xlwings as xw
import os
import time

def safe_refresh_excel(file_path):
    try:
        # Закрываем все экземпляры Excel
        os.system('taskkill /IM EXCEL.EXE /F')
        time.sleep(2)

        # Создаем временную копию
        temp_file = file_path.replace('.xlsx', '_temp.xlsx')
        if os.path.exists(temp_file):
            os.remove(temp_file)

        # Работаем через xlwings
        with xw.App(visible=True) as app:
            wb = xw.Book(file_path)

            # Обновляем данные
            wb.api.RefreshAll()
            time.sleep(30)  # Ожидание обновления

            # Сохраняем БЕЗ диалога восстановления
            wb.save(temp_file)
            wb.close()

            # Заменяем оригинальный файл
            os.remove(file_path)
            os.rename(temp_file, file_path)

        print("Файл успешно обновлен без диалога восстановления")

    except Exception as e:
        print(f"Ошибка: {str(e)}")
        if os.path.exists(temp_file):
            os.rename(temp_file, file_path)
    finally:
        os.system('taskkill /IM EXCEL.EXE /F')

# Использование
file_path = r'T:\Лимиты\_Модель данных\01_Реестр ССР\01_Реестр ССР.xlsx'
safe_refresh_excel(file_path)


Файл успешно обновлен без диалога восстановления


ОБНОВЛЕНИЕ ФАЙЛА 02_ТДЦ

In [3]:
import xlwings as xw
import os
import time
from datetime import datetime, timedelta
import pyautogui

def safe_refresh_excel(file_path, timeout_seconds=70):
    try:
        # Закрываем все экземпляры Excel
        os.system('taskkill /IM EXCEL.EXE /F')
        time.sleep(3)  # Увеличили задержку

        # Работаем через xlwings с явными параметрами
        app = xw.App(visible=True)
        app.display_alerts = False  # Отключаем предупреждения
        app.screen_updating = True  # Включаем обновление экрана

        print(f"Открываем файл: {file_path}")
        wb = app.books.open(file_path)

        # 1. Обновляем все данные
        print("Обновление всех подключений...")
        wb.api.RefreshAll()

        # Ожидание с проверкой состояния
        start_time = datetime.now()
        while (datetime.now() - start_time) < timedelta(seconds=timeout_seconds):
            try:
                if not wb.api.Calculating:
                    print("Основное обновление завершено")
                    break
                time.sleep(5)  # Увеличили интервал проверки
                print("Ожидаем завершения обновления...")
            except:
                continue

        # 2. Нажимаем кнопку макроса
        try:
            print("Поиск кнопки 'Обновить перечень этапов'...")
            sheet = wb.sheets['Управление']
            for shape in sheet.shapes:
                if 'Обновить перечень этапов' in shape.name or shape.text == 'Обновить перечень этапов':
                    print(f"Найдена кнопка: {shape.name}")
                    shape.api.ControlFormat.Execute()
                    print("Кнопка нажата, ожидаем выполнения...")
                    time.sleep(15)  # Увеличили время ожидания макроса
                    break
        except Exception as macro_error:
            print(f"Ошибка при выполнении макроса: {macro_error}")

        # 3. Сохраняем
        print("Сохранение файла...")
        wb.save()
        wb.close()
        app.quit()

        print("Файл успешно обновлен и сохранен")
        return True

    except Exception as e:
        print(f"Критическая ошибка: {str(e)}")
        return False
    finally:
        # Гарантированное закрытие
        try:
            if 'wb' in locals():
                wb.close()
            if 'app' in locals():
                app.quit()
        except:
            pass
        os.system('taskkill /IM EXCEL.EXE /F')

# Пример использования
if __name__ == "__main__":
    file_path = r'T:\Лимиты\_Модель данных\02_ТДЦ\_Управление\_Управление.xlsm'
    success = safe_refresh_excel(file_path)

    if success:
        print("Операция завершена успешно")
    else:
        print("Возникли проблемы при обновлении")



Открываем файл: T:\Лимиты\_Модель данных\02_ТДЦ\_Управление\_Управление.xlsm
Обновление всех подключений...
Поиск кнопки 'Обновить перечень этапов'...
Ошибка при выполнении макроса: (-2147352567, 'Ошибка.', (0, None, None, None, 0, -2147352565), None)
Сохранение файла...
Файл успешно обновлен и сохранен
Операция завершена успешно


ОБНОВЛЕНИЕ ФАЙЛА 05_ГОД ВВОДА

In [4]:
import xlwings as xw
import os
import time
from datetime import datetime, timedelta
import pyautogui

def safe_refresh_excel(file_path, timeout_seconds=70):
    try:
        # Закрываем все экземпляры Excel
        os.system('taskkill /IM EXCEL.EXE /F')
        time.sleep(3)  # Увеличили задержку

        # Работаем через xlwings с явными параметрами
        app = xw.App(visible=True)
        app.display_alerts = False  # Отключаем предупреждения
        app.screen_updating = True  # Включаем обновление экрана

        print(f"Открываем файл: {file_path}")
        wb = app.books.open(file_path)

        # 1. Обновляем все данные
        print("Обновление всех подключений...")
        wb.api.RefreshAll()

        # Ожидание с проверкой состояния
        start_time = datetime.now()
        while (datetime.now() - start_time) < timedelta(seconds=timeout_seconds):
            try:
                if not wb.api.Calculating:
                    print("Основное обновление завершено")
                    break
                time.sleep(5)  # Увеличили интервал проверки
                print("Ожидаем завершения обновления...")
            except:
                continue

        # 2. Нажимаем кнопку макроса
        try:
            print("Поиск кнопки 'Обновить перечень этапов'...")
            sheet = wb.sheets['Управление']
            for shape in sheet.shapes:
                if 'Обновить перечень этапов' in shape.name or shape.text == 'Обновить перечень этапов':
                    print(f"Найдена кнопка: {shape.name}")
                    shape.api.ControlFormat.Execute()
                    print("Кнопка нажата, ожидаем выполнения...")
                    time.sleep(15)  # Увеличили время ожидания макроса
                    break
        except Exception as macro_error:
            print(f"Ошибка при выполнении макроса: {macro_error}")

        # 3. Сохраняем
        print("Сохранение файла...")
        wb.save()
        wb.close()
        app.quit()

        print("Файл успешно обновлен и сохранен")
        return True

    except Exception as e:
        print(f"Критическая ошибка: {str(e)}")
        return False
    finally:
        # Гарантированное закрытие
        try:
            if 'wb' in locals():
                wb.close()
            if 'app' in locals():
                app.quit()
        except:
            pass
        os.system('taskkill /IM EXCEL.EXE /F')

# Пример использования
if __name__ == "__main__":
    file_path = r'T:\Лимиты\_Модель данных\05_Год ввода\_Управление\_Управление.xlsm'
    success = safe_refresh_excel(file_path)

    if success:
        print("Операция завершена успешно")
    else:
        print("Возникли проблемы при обновлении")

Открываем файл: T:\Лимиты\_Модель данных\05_Год ввода\_Управление\_Управление.xlsm
Обновление всех подключений...
Поиск кнопки 'Обновить перечень этапов'...
Ошибка при выполнении макроса: (-2147352567, 'Ошибка.', (0, None, None, None, 0, -2147352565), None)
Сохранение файла...
Файл успешно обновлен и сохранен
Операция завершена успешно


ОБНОВЛЕНИЕ ФАЙЛА 08_СТАТУС ПРОХОЖДЕНИЯ МГЭ

In [ ]:
import xlwings as xw
import os
import time

def safe_refresh_excel(file_path):
    try:
        # Закрываем все экземпляры Excel
        os.system('taskkill /IM EXCEL.EXE /F')
        time.sleep(2)

        # Создаем временную копию
        temp_file = file_path.replace('.xlsm', '_temp.xlsm')
        if os.path.exists(temp_file):
            os.remove(temp_file)

        # Работаем через xlwings
        with xw.App(visible=True) as app:
            wb = xw.Book(file_path)

            # Обновляем данные
            wb.api.RefreshAll()
            time.sleep(30)  # Ожидание обновления

            # Сохраняем БЕЗ диалога восстановления
            wb.save(temp_file)
            wb.close()

            # Заменяем оригинальный файл
            os.remove(file_path)
            os.rename(temp_file, file_path)

        print("Файл успешно обновлен без диалога восстановления")

    except Exception as e:
        print(f"Ошибка: {str(e)}")
        if os.path.exists(temp_file):
            os.rename(temp_file, file_path)
    finally:
        os.system('taskkill /IM EXCEL.EXE /F')

# Использование
file_path = r'T:\Лимиты\_Модель данных\08_Статус прохождения МГЭ\_Управление\_Управление.xlsm'
safe_refresh_excel(file_path)

Копирует файлы из нескольких исходных папок в архивную папку с датой

In [ ]:
import os
import shutil
from datetime import datetime
import logging

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='file_copy.log'
)

def copy_files_with_backup(source_folders, destination_base):
    """
    Копирует файлы из нескольких исходных папок в архивную папку с датой
    :param source_folders: список исходных папок
    :param destination_base: базовая папка для архива
    """
    try:
        # Создаем папку архива с текущей датой
        current_date = datetime.now().strftime('%Y-%m-%d')
        destination_folder = os.path.join(destination_base, current_date)

        os.makedirs(destination_folder, exist_ok=True)
        logging.info(f"Создана папка архива: {destination_folder}")

        total_copied = 0

        for source_folder in source_folders:
            if not os.path.exists(source_folder):
                logging.warning(f"Исходная папка не существует: {source_folder}")
                continue

            logging.info(f"Обработка папки: {source_folder}")

            for filename in os.listdir(source_folder):
                if filename.lower().endswith(('.xlsx', '.xlsm', '.xls', 'XLSX')):
                    source_file = os.path.join(source_folder, filename)
                    destination_file = os.path.join(destination_folder, filename)

                    try:
                        # Проверяем, не существует ли уже файл в архиве
                        if os.path.exists(destination_file):
                            # Добавляем timestamp к имени файла
                            timestamp = datetime.now().strftime('%H-%M-%S')
                            base, ext = os.path.splitext(filename)
                            new_filename = f"{base}_{timestamp}{ext}"
                            destination_file = os.path.join(destination_folder, new_filename)

                        shutil.copy2(source_file, destination_file)
                        total_copied += 1
                        logging.info(f"Скопирован: {filename} -> {os.path.basename(destination_file)}")

                    except Exception as copy_error:
                        logging.error(f"Ошибка при копировании {filename}: {str(copy_error)}")

        logging.info(f"Готово! Скопировано файлов: {total_copied}")
        return True

    except Exception as e:
        logging.critical(f"Критическая ошибка: {str(e)}")
        return False

# Пути для обработки (можно добавить сколько угодно папок)
source_folders = [
    r'T:\BIM\99. Прочее\Хохулин\Для обновления\01_Реестр ССР',
    r'T:\BIM\99. Прочее\Хохулин\Для обновления\02_ТДЦ',
    r'T:\BIM\99. Прочее\Хохулин\Для обновления\02_ТДЦ\_Управление',
    r'T:\BIM\99. Прочее\Хохулин\Для обновления\05_Год ввода',
    r'T:\BIM\99. Прочее\Хохулин\Для обновления\05_Год ввода\_Управление',
]

destination_base = r'T:\BIM\99. Прочее\Хохулин\Для обновления\Архив'

# Запуск копирования
if copy_files_with_backup(source_folders, destination_base):
    print("Операция завершена успешно. Подробности в file_copy.log")
else:
    print("Возникли ошибки при копировании. Проверьте file_copy.log")

СОЗДАТЬ БОЛЬШОЙ ФАЙЛ

In [35]:
#СОЗДАТЬ БОЛЬШОЙ ФАЙЛ


import pandas as pd
import os
from datetime import date
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm
from tqdm import tqdm



# Исключаемые файлы и папки
excluded_files = ['.XLSX.XLSX', 'ТЕСТ.XLSX']
excluded_folders = ['_Управление', 'Архив', 'новая форма']

# Список для хранения DataFrame
combined_dfs = []

# Укажите столбцы, которые хотите сохранить как строки (dtype_map и str "str тип данных" соблюдение формата данных как в исходниках)
dtype_map = {

    'Номер документа': str,
    'Стадия': str,
    'Ревизия': str,
    'Номер изменения': str,
    'Тип документа': str,
    'Проект': str,
    'Этап': str,
    'Объект': str,
    'Подобъект': str,
    'Получение документа от Разработчика - Дата (План)': str,
    'Получение документа от Разработчика - Дата (Прогноз)': str,
    'Получение документа от Разработчика - Дата (Факт)': str,
    'Статус': str,
    'Статус Заказчика': str,
    'Статус МИП': str,
    'Документ передан в МИП в окончательной редакции в бумажном виде – Дата (План)': str,
    'Документ передан в МИП в окончательной редакции в бумажном виде – Дата (Прогноз)': str,
    'Выдача документа в производство работ - Дата (Факт) (временный)': str,
    'Марка': str,
    'Период': str,
    'Веб ссылка': str

}

def make_links_non_clickable(text):
    """Преобразует ссылки в обычный текст"""
    if pd.isna(text):
        return text

    text = str(text)
    # # Заменяем http:// и https:// на пустую строку
    text = text.replace('http://', '').replace('https://', '')
    # Удаляем специальные символы, которые могут сделать текст кликабельным
    text = text.replace('[', '').replace(']', '').replace('<', '').replace('>', '')
    return text

# Обработка файлов
for filename in os.listdir(folder_path):
    if filename.endswith('.XLSX'):
        file_path = os.path.join(folder_path, filename)
        try:
            # Чтение данных
            combined_df = pd.read_excel(file_path, usecols=list(dtype_map.keys()), dtype=dtype_map)

            # Обработка столбца с ссылками
            if 'Веб ссылка' in combined_df.columns:
                combined_df['Веб ссылка'] = combined_df['Веб ссылка'].apply(make_links_non_clickable)

            combined_dfs.append(combined_df)

        except Exception as e:
            print(f"Ошибка при обработке файла {filename}: {str(e)}")


# Объединение всех DataFrame в один
combined_df = pd.concat(combined_dfs, ignore_index=True) #ПЛАН/ФАКТ ОБЩИЙ




# Запись объединенного DataFrame в новый файл Excel
with pd.ExcelWriter(r'T:\BIM\99. Прочее\Хохулин\09_ГРД_Большой для проверки.xlsx') as writer:
    combined_df.to_excel(writer, sheet_name="ПЛАН_ФАКТ_ОБЩИЙ", index=False)









IT_ОБЕСПЕЧЕНИЕ

In [ ]:
#Бюджет_П/Ф/Контрагент


import pandas as pd
import os
from datetime import date
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm
from tqdm import tqdm
import re
import glob

#Загрузите файлы Excel
file1_path = r'T:\BIM\3. Систематизация\02_Задачи\2025.07.07_Бюджет IT\из 1С\Бюджет.xls'
file2_path = r'T:\BIM\3. Систематизация\02_Задачи\2025.07.07_Бюджет IT\из 1С\Бюджет план -факт (3).xls'
file3_path = r'T:\BIM\3. Систематизация\02_Задачи\2025.07.07_Бюджет IT\из 1С\отчет.xlsx'
# combined_df = pd.read_excel(file3_path, sheet_name='TDSheet', skiprows=8)

# Чтение данных
df1 = pd.read_excel(file1_path, usecols=['Ответственный', 'Статья оборотов', 'Проект', 'Упр аналитика',
                                           'Количество', 'Январь', 'Февраль','Март', 'Апрель', 'Май',
                                           'Июнь', 'Июль','Август', 'Сентябрь','Октябрь', 'Ноябрь', 'Декабрь',
                                           'Итог'])
df2 = pd.read_excel(file2_path, usecols=['Упр аналитика', 'Факт'])

df3 = pd.read_excel(file3_path, usecols=['Закупка.Предмет закупки', 'Закупка.Договор контрагента.Контрагент', 'Закупка.Договор контрагента.Номер договора',
                                         'Закупка.Договор контрагента.Дата договора', 'Закупка.Договор контрагента.Сумма', 'Упр аналитика', 'Сумма'])


#Переименование столца
df1= df1.rename(columns={'Итог': 'План'})
df3= df3.rename(columns={'Закупка.Предмет закупки': 'Предмет закупки'})
df3= df3.rename(columns={'Закупка.Договор контрагента.Контрагент': 'Контрагент'})
df3= df3.rename(columns={'Закупка.Договор контрагента.Номер договора': 'Номер договора'})
df3= df3.rename(columns={'Закупка.Договор контрагента.Дата договора': 'Дата договора'})
df3= df3.rename(columns={'Закупка.Договор контрагента.Сумма': 'Сумма договора'})


# Объединение данных
combined_df = df1.merge(df2, on='Упр аналитика', how='left')
combined_df = combined_df.merge(df3, on='Упр аналитика', how='left')




# Сохранение в файл Excel
with pd.ExcelWriter(r'T:\BIM\3. Систематизация\02_Задачи\2025.07.07_Бюджет IT\из 1С\Новая папка\IT_Обеспечение.xlsx') as writer:
    combined_df.to_excel(writer, sheet_name="Обеспечение", index=False)

ВНЕ ГРД ИНСТИТУТ

In [ ]:
import pandas as pd
import os
from datetime import date
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm
from tqdm import tqdm
import re
from openpyxl import load_workbook

 #Путь к папке с файлами Excel
folder_path = r'T:\BIM\3. Систематизация\01_Проекты\01_ИБ Реестр ССР\05_Тест\ГРД\Пример ГРД'

# Список для хранения DataFrame
combined_dfs = []

# Перебор всех файлов в папке
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file_name)
        # Чтение данных из файла Excel (начиная с 5-й строки и листа ГРД_РД)
        combined_df = pd.read_excel(file_path, sheet_name='ГРД_РД',header=4, usecols=['Обозначение комплекта','Наименование комплекта', 'Проект'])
        combined_dfs.append(combined_df)


# Объединение всех DataFrame в один
combined_df = pd.concat(combined_dfs, ignore_index=True)

# Удаление строк, где "Номер документа" пустой
combined_df = combined_df[combined_df['Обозначение комплекта'].notna()]
combined_df = combined_df[combined_df['Проект'] != 21]

# Запись объединенного DataFrame в новый файл Excel
with pd.ExcelWriter(r'T:\BIM\3. Систематизация\01_Проекты\01_ИБ Реестр ССР\05_Тест\ГРД\Институт.xlsx') as writer:
    combined_df.to_excel(writer, sheet_name="Институт", index=False)


ВНЕ ГРД ОБЩИЙ

In [ ]:
import pandas as pd
import os
from datetime import date
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm
from tqdm import tqdm
import re
from openpyxl import load_workbook

# Путь к папке с файлами Excel
folder_path = r'T:\Лимиты\_Модель данных\09_ГРД'

# Исключаемые файлы и папки
excluded_files = ['.XLSX.XLSX', 'ТЕСТ.XLSX']
excluded_folders = ['_Управление', 'Архив', 'новая форма']

# Список для хранения DataFrame
dfs = []

# Перебор всех файлов в папке
for filename in os.listdir(folder_path):
    if filename.endswith('.XLSX'):
        filepath = os.path.join(folder_path, filename)
        # Чтение данных из файла Excel (начиная с 5-й строки и листа ГРД_РД)
        df = pd.read_excel(filepath, usecols=['Номер документа', 'Стадия'])
        dfs.append(df)

# Объединение всех DataFrame в один
combined_df = pd.concat(dfs, ignore_index=True)

# Фильтрация по столбцу "Стадия"
combined_df = combined_df[combined_df['Стадия'] == 'Рабочая документация']
#Фильтрация по столбцу Номер документа исключает Пусто
combined_df = combined_df[combined_df['Номер документа'].notna()]

#Фильтрация по столбцу Номер документа исключает позиции
combined_df = combined_df[combined_df['Номер документа'] != 'Электродепо Сокол-126-03-ОМ.01']
combined_df = combined_df[combined_df['Номер документа'] != 'Печать из архива (МТЛН)']
combined_df = combined_df[combined_df['Номер документа'] != 'тест иерархии']
combined_df = combined_df[combined_df['Номер документа'] != 'Примыкание к ст."Сокольники" Сокольнической линии метрополитена. Архитектурные работы']
combined_df = combined_df[combined_df['Номер документа'] != 'Документация']
combined_df = combined_df[combined_df['Номер документа'] != 'Материалы']
combined_df = combined_df[combined_df['Номер документа'] != 'Досмотровая']
combined_df = combined_df[combined_df['Номер документа'] != 'Конструкция']
combined_df = combined_df[combined_df['Номер документа'] != 'Лист']
combined_df = combined_df[combined_df['Номер документа'] != 'Научно-проектная документация']
combined_df = combined_df[combined_df['Номер документа'] != 'План']
combined_df = combined_df[combined_df['Номер документа'] != 'Планы']
combined_df = combined_df[combined_df['Номер документа'] != 'ПОЯСНИТЕЛЬНАЯ']
combined_df = combined_df[combined_df['Номер документа'] != 'проект изменения']
combined_df = combined_df[combined_df['Номер документа'] != 'Расположение оборудования']
combined_df = combined_df[combined_df['Номер документа'] != 'Спецификация']
combined_df = combined_df[combined_df['Номер документа'] != 'Станция "Сокольники"']
combined_df = combined_df[combined_df['Номер документа'] != 'Технологический регламент']



# Запись объединенного DataFrame в новый файл Excel
with pd.ExcelWriter(r'T:\BIM\3. Систематизация\01_Проекты\01_ИБ Реестр ССР\05_Тест\ГРД\ГРД_Общий.xlsx') as writer:
    combined_df.to_excel(writer, sheet_name="ГРД", index=False)

ВНЕ ГРД ОБЪЕДИНЕННЫЙ

In [ ]:
import pandas as pd
import os
from datetime import datetime

import os
import pandas as pd

# Путь к первой папке с файлами Excel
folder_path_1 = r'T:\BIM\3. Систематизация\01_Проекты\01_ИБ Реестр ССР\05_Тест\ГРД\Пример ГРД'

# Список для хранения DataFrame из первой папки
combined_df_1 = []

# Перебор всех файлов в первой папке
for file_name in os.listdir(folder_path_1):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path_1, file_name)
        # Чтение данных из файла Excel (начиная с 5-й строки и листа ГРД_РД)
        df = pd.read_excel(file_path, sheet_name='ГРД_РД', header=4,
                           usecols=['Обозначение комплекта', 'Наименование комплекта', 'Проект'])
        combined_df_1.append(df)

# Объединение всех DataFrame в один для первой папки
final_df_1 = pd.concat(combined_df_1, ignore_index=True)

# Путь ко второй папке с файлами Excel
folder_path_2 = r'T:\Лимиты\_Модель данных\09_ГРД'

# Исключаемые файлы
excluded_files = ['.XLSX.XLSX', 'ТЕСТ.XLSX']
# Список для хранения DataFrame из второй папки
combined_df_2 = []

# Перебор всех файлов во второй папке
for filename in os.listdir(folder_path_2):
    if filename.endswith('.XLSX') and filename not in excluded_files:
        filepath = os.path.join(folder_path_2, filename)
        # Чтение данных из файла Excel
        df = pd.read_excel(filepath, usecols=['Номер документа', 'Стадия'])
        combined_df_2.append(df)

# Объединение всех DataFrame в один для второй папки
final_df_2 = pd.concat(combined_df_2, ignore_index=True)

# Фильтрация по столбцу "Стадия"
final_df_2 = final_df_2[final_df_2['Стадия'] == 'Рабочая документация']
final_df_2 = final_df_2[final_df_2['Номер документа'].notna()]

# Исключение нежелательных значений
excluded_values = ['Электродепо Сокол-126-03-ОМ.01', 'Печать из архива (МТЛН)',
                   'тест иерархии', 'Примыкание к ст."Сокольники" Сокольнической линии метрополитена. Архитектурные работы',
                   'Документация', 'Материалы', 'Досмотровая', 'Конструкция', 'Лист',
                   'Научно-проектная документация', 'План', 'Планы', 'ПОЯСНИТЕЛЬНАЯ',
                   'проект изменения', 'Расположение оборудования', 'Спецификация',
                   'Станция "Сокольники"', 'Технологический регламент', 'Тест111', 'Тест222', 'Тест333','Архив РД со штампом CIBSI']

for value in excluded_values:
    final_df_2 = final_df_2[final_df_2['Номер документа'] != value]

# Удаление строк, где "Обозначение комплекта" пустой и "Проект" равен 21
final_df_1 = final_df_1[final_df_1['Обозначение комплекта'].notna()]
final_df_1 = final_df_1[final_df_1['Проект'] != 21]

# Удаление дубликатов по столбцу 'Номер документа'
final_df_2 = final_df_2 .drop_duplicates(subset='Номер документа')

#Сравнение двух столбцов если в ГРД есть значения из листа Институт то 0 в противном случае 1
# Создаем множество эталонных значений для точного сравнения
grd_values = set(final_df_1['Обозначение комплекта'].dropna().astype(str))
    # Добавляем столбец с правильной логикой:
    # 0 - если номер документа есть в ГРД (точное совпадение)
    # 1 - если номера нет в ГРД
final_df_2['Вне ГРД'] = final_df_2['Номер документа'].apply(
        lambda x: '0' if str(x) in grd_values else '1'
    )


# # Сравнение столбцов и добавление нового столбца
# final_df_2['Вне ГРД'] = 1  # По умолчанию 1 (нет совпадения)
# # Если значение из final_df_1 есть в final_df_2, ставим 0
# final_df_2.loc[final_df_2['Номер документа'].isin(final_df_1['Обозначение комплекта']), 'Вне ГРД'] = 0



# Запись объединенного DataFrame в новый файл Excel на разных листах
with pd.ExcelWriter(r'T:\BIM\3. Систематизация\01_Проекты\01_ИБ Реестр ССР\05_Тест\ГРД\ИМИП_ГРД..xlsx') as writer:
    final_df_1.to_excel(writer, sheet_name="Институт", index=False)
    final_df_2.to_excel(writer, sheet_name="ГРД", index=False)
